In [132]:
%load_ext autoreload
%autoreload 2

import os

import tqdm
from rich import print
from rich.traceback import install

import datasets


os.environ["HYDRA_FULL_ERROR"] = "1"
install()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f9631a1fc40>>

In [133]:
# ! pip install -e ../
# ! mamba install -c conda-forge itables pandas -y
# ! pip install pytorchvideo
import copy
import json
import os
import pathlib
import random
import sys
import time
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from dataclasses import dataclass
from enum import Enum
from math import floor
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import datasets

import numpy as np
import pandas as pd
import PIL
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from rich import print
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from torchvision.transforms._transforms_video import CenterCropVideo
from traitlets import default
from transformers import CLIPModel, CLIPProcessor
import datasets
from tali_wit.data import (
    AnyModalSample,
    dataclass_collate,
    default_image_transforms,
    ModalityTypes,
    select_subtitles_between_timestamps,
    TALIDataset,
)
from tqdm.notebook import tqdm



In [134]:
from tali_wit.decorators import configurable
from tali_wit.utils import get_logger, load_json, save_json
from tali_wit.models import ModalityConfig

logger = get_logger(__name__)


In [135]:
from tali_wit.data_plus import TALIBaseDemoTransform, TALIBaseTransformConfig


transform = TALIBaseDemoTransform(
    config=TALIBaseTransformConfig(
        root_filepath="/data/datasets/tali-wit-2-1-buckets/",
        modality_list=[
            ModalityTypes.wit_image.value,
            ModalityTypes.wit_caption.value,
            ModalityTypes.wit_title.value,
            ModalityTypes.wit_main_body.value,
            ModalityTypes.youtube_image.value,
            ModalityTypes.youtube_video.value,
            ModalityTypes.youtube_subtitles.value,
            ModalityTypes.youtube_audio.value,
            ModalityTypes.youtube_description.value,
        ],
        rng_seed=42,
        top_k_tali=10,
        image_size=224,
        num_video_frames=200,
        num_audio_frames=160000,
        clip_duration_in_seconds=10.0,
        deterministic_sampling=True,
    )
)
train_dataset = datasets.load_from_disk("/data/datasets/tali-wit-2-1-buckets/train-set")
train_dataset = train_dataset.with_transform(transform)
val_dataset = datasets.load_from_disk("/data/datasets/tali-wit-2-1-buckets/val-set")
val_dataset = val_dataset.with_transform(transform)
test_dataset = datasets.load_from_disk("/data/datasets/tali-wit-2-1-buckets/test-set")
test_dataset = test_dataset.with_transform(transform)
num_samples = 0

# for idx, sample in enumerate(tqdm(dataset)):
#     output = {key: value.shape if isinstance(value, torch.Tensor) else value for key, value in sample.items()}
#     print(output)
        
#     if idx >= num_samples:
#         break

In [136]:
# ! pip install gradio
from collections import defaultdict
from distutils.command.upload import upload
import gradio as gr

In [137]:

# {
#     'wit_idx': 502620,
#     'wikipedia_caption_image': torch.Size([3, 224, 224]),
#     'wikipedia_text': '<section_title> Station layout </section_title>',
#     'youtube_video_id': '6e7RO-o6u6w',
#     'youtube_content_video': torch.Size([10, 3, 224, 224]),
#     'youtube_content_audio': torch.Size([16000]),
#     'youtube_description_text': "<ysub> it's open somebody's gonna be building something there so the neighborhood 
# isn't is in change i don't think shintomicho has much of a personality when they took away the kabuki theater it 
# really did change  </ysub>"
# }
from collections import defaultdict
from distutils.command.upload import upload
from importlib.resources import path
import gradio as gr
import torchvision
import torchaudio

dataset_dict = {"train": train_dataset, "val": val_dataset, "test": test_dataset}

def update_length_options(set_name):
    max_idx = len(dataset_dict[set_name]) - 1
    return gr.update(minimum=0, maximum=max_idx, step=1)

def get_random_sample(set_name):
    dataset = dataset_dict[set_name]
    sample_index = random.randint(0, len(dataset) - 1)
    return sample_index

def generate_caption_output(caption_dict):
    with gr.Column() as output:
        for language_key, language_captions in caption_dict.items():
            with gr.Tab(language_key):
                for caption_key, caption in language_captions.items():
                    gr.Textbox(value=caption, label=caption_key)
                
    return gr.update(children=[output])

def update_captions(language, set_name, sample_index):
    dataset = dataset_dict[set_name]
    sample = dataset[int(sample_index)]
    caption_dict = sample["captions"][language]
    
    for key in [ "caption_alt_text_description",
                "caption_reference_description",
                "caption_title_and_reference_description",
                "context_page_description",
                "context_section_description",
                "hierarchical_section_title",
                "page_title",
                "section_title",
            ]:
        if key not in caption_dict:
            caption_dict[key] = "<Unavailable/>"
    
    
    return [gr.update(value=caption_dict["caption_alt_text_description"]), 
            gr.update(value=caption_dict["caption_reference_description"]),
            gr.update(value=caption_dict["caption_title_and_reference_description"]),
            gr.update(value=caption_dict["context_page_description"]),
            gr.update(value=caption_dict["context_section_description"]),
            gr.update(value=caption_dict["hierarchical_section_title"]),
            gr.update(value=caption_dict["page_title"]),
            gr.update(value=caption_dict["section_title"])]
def update_language_choices(set_name, sample_index):
    print(dataset_dict[set_name][int(sample_index)])
    languages = list(dataset_dict[set_name][int(sample_index)]["captions"].keys())
    return gr.update(choices=languages, value=languages[0]), *update_captions(languages[0], set_name, sample_index)
def load_sample(set_name, sample_index):
    # Load the dataset based on the set name (you'll need to implement this part)
    dataset = dataset_dict[set_name]

    # Retrieve the sample at the given index
    sample = dataset[int(sample_index)]
    # Extract the text, image, video, and audio from the sample (you'll need to adapt this to your specific dataset)
    print(sample)
    subtitles = sample["youtube_description_text"]
    wit_image = sample["wikipedia_caption_image"].squeeze().permute(1, 2, 0).numpy()
    youtube_image = sample["youtube_random_video_sample_image"].squeeze().permute(1, 2, 0).numpy()
    video = sample["youtube_content_video"].squeeze().permute(0, 2, 3, 1).numpy() * 255
    audio = sample["youtube_content_audio"]
    
    video_path = f"../demo/temp_data/video-{set_name}-{sample_index}.mp4"
    audio_path = f"../demo/temp_data/audio-{set_name}-{sample_index}.mp3"
    if not pathlib.Path(video_path).parent.exists():
        pathlib.Path(video_path).parent.mkdir(parents=True, exist_ok=True)
    print(audio.shape)
    if not pathlib.Path(video_path).exists():
        torchvision.io.write_video(video_path, video, fps=20)
    if not pathlib.Path(audio_path).exists():
        torchaudio.save(audio_path, audio.view(-1).unsqueeze(0), 16000)
    return *update_language_choices(set_name=set_name, sample_index=sample_index), subtitles, wit_image, youtube_image, video_path, audio_path

def load_random_sample(set_name):
    sample_idx = get_random_sample(set_name)
    return gr.update(value=sample_idx), *load_sample(set_name, sample_idx)

In [138]:
# import concurrent.futures

# for set_name in dataset_dict.keys():
#     with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
#         with tqdm(total=len(dataset_dict[set_name])) as pbar:
#             for _ in executor.map(load_sample, [set_name] * len(dataset_dict[set_name]), range(len(dataset_dict[set_name]))):
#                 pbar.update(1)

In [139]:

callback = gr.CSVLogger()

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # TALI (Temporally and semantically Aligned Audio, Language and Images) Dataset Demo v-0.3.0 🖼️ 🔊 🎦 📝
    ## What should I expect to see here? 
    This demo is intended to show you what the TALI dataset looks like. It is a dataset that used the [Wikipedia Image Text (WIT)](https://huggingface.co/datasets/wikimedia/wit_base) captions and article titles to search Youtube for videos that match the captions, and then subsquently downloads the video, audio, and subtitles from such videos.
    The result is a rich multi modal dataset that has multiple caption types related to both the WiT Images, and the Youtube videos. This means learning can take place between either temporally or semantically aligned text, images, audio and video.
    ## How was this collected?
    1. We start from the [WiT dataset](https://huggingface.co/datasets/wikimedia/wit_base) use either the context_page_description or page_title, which we refer to as source-query, search youtube with it. Return top 100 result titles.
    2. Compare the returning titles, which we'll call youtube-titles, with the source-query using the CLIP text embeddings of the largest CLIP model (patch-14, large) we had available whose alignments were the closest we've seen with human perception.
    3. Choose top-1 title’s video based on the CLIP ranking.
    4. Download video, break into 30 second segments. Apply CLIP image embedding to the first image of each segment, and compare with the video’s title text. Rank the segments based on this distance.
    5. Choose the top-10 segments for each video. Extract image, audio and subtitle frames.
    At sampling time:
    Randomly select one of these 10 segments, choose a 10 second segment out of the 30 second clip. Return 200 video frames (spread throughout the 10 second segment), and, 160000 audio frames (10 seconds).
    """)
    
    gr.Markdown(
    """
    ### First select the set to sample from, and the index of the sample to load, and click the "Fetch sample" button OR simply click the "Fetch random sample" button to load a random sample.
    """)
    with gr.Row():
        with gr.Column():
            input_set_name = gr.Dropdown(choices=["train", "val", "test"], value="train", label="Set name", info="Select the set to sample from")
        with gr.Column():
            input_sample_index = gr.Slider(minimum=0, maximum=100, randomize=True, step=1, interactive=True,
                label="Datapoint idx to sample",
                info="Select the idx to sample",
            )
        with gr.Column():
            fetch_btn = gr.Button("Fetch sample")
            fetch_random_btn = gr.Button("Fetch random sample")
            
            
    input_set_name.change(update_length_options, input_set_name, input_sample_index)
    gr.Markdown(
    """
    ### The wikipedia image is semantically aligned to the youtube components, while the youtube components are temporally aligned to each other.
    """)
    output_subtitle = gr.Text(label="Youtube Subtitles")
    caption_title_and_reference_description = gr.Textbox(label="caption_title_and_reference_description")
    page_title = gr.Textbox(label="page_title")
    with gr.Row():
        with gr.Column():
            output_wit_image = gr.Image(label="Wikipedia Image")
        with gr.Column():
            output_youtube_image = gr.Image(label="Youtube Image")
        with gr.Column():
            output_video = gr.Video(label="Youtube Video")
        with gr.Column():
            output_audio = gr.Audio(label="Youtube Audio")
    
    gr.Markdown(
    """
    ### Choose what language to display captions in (the captions are in multiple languages)
    """)
    output_language = gr.Dropdown(label="Wiki language ID")
    
    gr.Markdown(
    """
    ### These captions are semantically aligned to the wikipedia image, and should ideally be semantically aligned to the youtube components, however the dataset was selected automatically and this is not always the case. Overall however, the captions are very good at describing the youtube components.
    """)
    
    with gr.Row():
        section_title = gr.Textbox(label="section_title")
        hierarchical_section_title = gr.Textbox(label="hierarchical_section_title")
    with gr.Row():
        caption_alt_text_description = gr.Textbox(label="caption_alt_text_description")
        caption_reference_description = gr.Textbox(label="caption_reference_description")
    with gr.Row():
        context_section_description = gr.Textbox(label="context_section_description")
        context_page_description = gr.Textbox(label="context_page_description")    
    
        
    output_language.change(update_captions, [output_language, input_set_name, input_sample_index], [caption_alt_text_description, caption_reference_description, caption_title_and_reference_description, context_page_description, context_section_description, hierarchical_section_title, page_title, section_title])
    
    
    report_textbox = gr.Textbox(info="Please describe the issue you found with the sample", label="Issue description")
    callback.setup([input_set_name, input_sample_index, output_language, report_textbox], "flagged_data_points")
    report_button = gr.Button("Report Issue", info="Report an issue with the sample")
    report_button.click(lambda *args: callback.flag(args), [input_set_name, input_sample_index, output_language, report_textbox], None, preprocess=False)
    report_button.click(lambda x, y: [gr.update(value="Issue has been reported. Thank you for your help!", interactive=False), gr.update(value="Issue has been reported. Thank you for your help!", visible=False)], [report_button, report_textbox], [report_button, report_textbox])
    
    # fetch_random_btn.click(update_language_choices, [input_set_name, input_sample_index], [output_language, caption_alt_text_description, caption_reference_description, caption_title_and_reference_description, context_page_description, context_section_description, hierarchical_section_title, page_title, section_title])
    fetch_random_btn.click(fn=load_random_sample,
    inputs=[input_set_name],
    outputs=[input_sample_index, output_language, caption_alt_text_description,
            caption_reference_description,
            caption_title_and_reference_description,
            context_page_description,
            context_section_description,
            hierarchical_section_title,
            page_title,
            section_title,
            output_subtitle, output_wit_image, output_youtube_image, output_video, output_audio],)
    
    # fetch_btn.click(update_language_choices, [input_set_name, input_sample_index], [output_language, caption_alt_text_description, caption_reference_description, caption_title_and_reference_description, context_page_description, context_section_description, hierarchical_section_title, page_title, section_title])
    fetch_btn.click(fn=load_sample,
    inputs=[input_set_name, input_sample_index],
    outputs=[output_language, caption_alt_text_description,
            caption_reference_description,
            caption_title_and_reference_description,
            context_page_description,
            context_section_description,
            hierarchical_section_title,
            page_title,
            section_title,
            output_subtitle, output_wit_image, output_youtube_image, output_video, output_audio],)


In [140]:
demo.queue(concurrency_count=8)
demo.launch(share=True, debug=True, enable_queue=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://bcddf7d7fbc6cde520.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


{
    'wit_idx': 345184,
    'captions': {
        'en': {
            'caption_reference_description': 'The uranium-238 series is a series of α (N and Z less 2) and β− 
decays (N less 1, Z plus 1) to nuclides that are successively deeper into the valley of stability. The series 
terminates at lead-206, a stable nuclide at the bottom of the valley of stability.',
            'context_page_description': 'In nuclear physics, the valley of stability is a characterization of the 
stability of nuclides to radioactivity based on their binding energy. Nuclides are composed of protons and 
neutrons. The shape of the valley refers to the profile of binding energy as a function of the numbers of neutrons 
and protons, with the lowest part of the valley corresponding to the region of most stable nuclei. The line of 
stable nuclides down the center of the valley of stability is known as the line of beta stability. The sides of the
valley correspond to increasing instability to beta decay. The decay of a nuclide becomes more energetically 
favorable the further it is from the line of beta stability. The boundaries of the valley correspond to the nuclear
drip lines, where nuclides become so unstable they emit single protons or single neutrons. Regions of instability 
within the valley at high atomic number also include radioactive decay by alpha radiation or spontaneous fission. 
The shape of the valley is roughly an elongated paraboloid corresponding to the nuclide binding energies as a 
function of neutron and atomic numbers.',
            'context_section_description': 'The valley of stability can be helpful in interpreting and 
understanding properties of nuclear decay processes such as decay chains and nuclear fission.',
            'hierarchical_section_title': 'Valley of stability / Discussion',
            'page_title': 'Valley of stability',
            'section_title': 'Discussion'
        },
        'bn': {
            'caption_reference_description': 'ইউরেনিয়াম-২৩৮ ধারা হল একটি আলফা (N এবং Z - ২) এবং বিটা (N - ১, Z + ১)
ক্ষয়ের ধারা, যার মধ্য দিয়ে নিউক্লিয়াসটি ধাপে ধাপে স্থায়িত্ব উপত্যকার গভীরে প্রবেশ করে, এবং সবশেষে Pb-২০৬ এ পৌঁছায়, যার
অবস্থান স্থায়িত্ব উপত্যকার তলদেশে।',
            'context_page_description': 'স্থায়িত্ব উপত্যকা বা valley of stability হল তেজস্ক্রিয়তার সাপেক্ষে বন্ধন শক্তির
ভিত্তিতে নিউক্লাইডের স্থায়িত্বের বৈশিষ্ট্য। উপত্যকাটির গড়পড়তা আকৃতি দীর্ঘায়িত উপবৃত্তাকার, যা নিউট্রন ও প্রোটন সংখ্যার ফাংশন
হিসেবে বন্ধন শক্তির একটি চিত্র গঠন করে। উপত্যকার গভীরতর এলাকায় স্থিতিশীল নিউক্লাইডের অবস্থান। উপত্যকার মধ্যভাগ বরাবর অবস্থিত
স্থিতিশীল নিউক্লাইডের সারিকে বিটা স্থায়িত্ব রেখা বলা হয়। উপত্যকার দুই প্রান্তের দিকে বিটা ক্ষয় জনিত ক্রমাগত উচ্চতর অস্থিতিশীলতা
পাওয়া যায়। স্থায়িত্ব উপত্যকায় কোন নিউক্লাইড বিটা স্থায়িত্ব রেখা থেকে যত দূরে অবস্থিত, তার অস্থায়িত্বের সম্ভাবনা তত বেশি।
উপত্যকার সীমানা অঞ্চল পারমাণবিক ক্ষরণ রেখার সাথে সম্পর্কিত, যেখানে নিউক্লাইডগুলো এতটা অস্থায়ী যে একক প্রোটন এবং একক নিউট্রন
বিকিরণ করতে থাকে। উপত্যকার অভ্যন্তরে উচ্চ পারমাণবিক সংখ্যা এলাকায় আলফা ক্ষয় বা স্বত:স্ফূর্ত ফিশন জনিত অস্থায়িত্ব দেখা যায়।',
            'hierarchical_section_title': 'স্থায়িত্ব উপত্যকা / আলোচনা',
            'page_title': 'স্থায়িত্ব উপত্যকা',
            'section_title': 'আলোচনা'
        }
    },
    'wikipedia_caption_image': tensor([[[0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9451, 0.9451, 0.9451,  ..., 0.9686, 0.9647, 0.9647],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[0.9961, 0.9961, 0.9961,  ..., 0.9961, 0.9961, 0.9961],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9451, 0.9451, 0.9451,  ..., 0.9725, 0.9725, 0.9725],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
      

torch.Size([160000])

{
    'wit_idx': 345184,
    'captions': {
        'en': {
            'caption_reference_description': 'The uranium-238 series is a series of α (N and Z less 2) and β− 
decays (N less 1, Z plus 1) to nuclides that are successively deeper into the valley of stability. The series 
terminates at lead-206, a stable nuclide at the bottom of the valley of stability.',
            'context_page_description': 'In nuclear physics, the valley of stability is a characterization of the 
stability of nuclides to radioactivity based on their binding energy. Nuclides are composed of protons and 
neutrons. The shape of the valley refers to the profile of binding energy as a function of the numbers of neutrons 
and protons, with the lowest part of the valley corresponding to the region of most stable nuclei. The line of 
stable nuclides down the center of the valley of stability is known as the line of beta stability. The sides of the
valley correspond to increasing instability to beta decay. The decay of a nuclide becomes more energetically 
favorable the further it is from the line of beta stability. The boundaries of the valley correspond to the nuclear
drip lines, where nuclides become so unstable they emit single protons or single neutrons. Regions of instability 
within the valley at high atomic number also include radioactive decay by alpha radiation or spontaneous fission. 
The shape of the valley is roughly an elongated paraboloid corresponding to the nuclide binding energies as a 
function of neutron and atomic numbers.',
            'context_section_description': 'The valley of stability can be helpful in interpreting and 
understanding properties of nuclear decay processes such as decay chains and nuclear fission.',
            'hierarchical_section_title': 'Valley of stability / Discussion',
            'page_title': 'Valley of stability',
            'section_title': 'Discussion'
        },
        'bn': {
            'caption_reference_description': 'ইউরেনিয়াম-২৩৮ ধারা হল একটি আলফা (N এবং Z - ২) এবং বিটা (N - ১, Z + ১)
ক্ষয়ের ধারা, যার মধ্য দিয়ে নিউক্লিয়াসটি ধাপে ধাপে স্থায়িত্ব উপত্যকার গভীরে প্রবেশ করে, এবং সবশেষে Pb-২০৬ এ পৌঁছায়, যার
অবস্থান স্থায়িত্ব উপত্যকার তলদেশে।',
            'context_page_description': 'স্থায়িত্ব উপত্যকা বা valley of stability হল তেজস্ক্রিয়তার সাপেক্ষে বন্ধন শক্তির
ভিত্তিতে নিউক্লাইডের স্থায়িত্বের বৈশিষ্ট্য। উপত্যকাটির গড়পড়তা আকৃতি দীর্ঘায়িত উপবৃত্তাকার, যা নিউট্রন ও প্রোটন সংখ্যার ফাংশন
হিসেবে বন্ধন শক্তির একটি চিত্র গঠন করে। উপত্যকার গভীরতর এলাকায় স্থিতিশীল নিউক্লাইডের অবস্থান। উপত্যকার মধ্যভাগ বরাবর অবস্থিত
স্থিতিশীল নিউক্লাইডের সারিকে বিটা স্থায়িত্ব রেখা বলা হয়। উপত্যকার দুই প্রান্তের দিকে বিটা ক্ষয় জনিত ক্রমাগত উচ্চতর অস্থিতিশীলতা
পাওয়া যায়। স্থায়িত্ব উপত্যকায় কোন নিউক্লাইড বিটা স্থায়িত্ব রেখা থেকে যত দূরে অবস্থিত, তার অস্থায়িত্বের সম্ভাবনা তত বেশি।
উপত্যকার সীমানা অঞ্চল পারমাণবিক ক্ষরণ রেখার সাথে সম্পর্কিত, যেখানে নিউক্লাইডগুলো এতটা অস্থায়ী যে একক প্রোটন এবং একক নিউট্রন
বিকিরণ করতে থাকে। উপত্যকার অভ্যন্তরে উচ্চ পারমাণবিক সংখ্যা এলাকায় আলফা ক্ষয় বা স্বত:স্ফূর্ত ফিশন জনিত অস্থায়িত্ব দেখা যায়।',
            'hierarchical_section_title': 'স্থায়িত্ব উপত্যকা / আলোচনা',
            'page_title': 'স্থায়িত্ব উপত্যকা',
            'section_title': 'আলোচনা'
        }
    },
    'wikipedia_caption_image': tensor([[[0.9961, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9451, 0.9451, 0.9451,  ..., 0.9961, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

        [[0.9961, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
         [0.9451, 0.9451, 0.9451,  ..., 0.9961, 1.0000, 1.0000],
         ...,
         [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
      

{
    'wit_idx': 16324,
    'captions': {
        'sv': {
            'caption_title_and_reference_description': 'Aechmea pimenti-velosoi [SEP] ',
            'context_page_description': 'Aechmea pimenti-velosoi är en gräsväxtart som beskrevs av Raulino Reitz. 
Aechmea pimenti-velosoi ingår i släktet Aechmea och familjen Bromeliaceae. Inga underarter finns listade i 
Catalogue of Life.',
            'context_section_description': 'Aechmea pimenti-velosoi är en gräsväxtart som beskrevs av Raulino 
Reitz. Aechmea pimenti-velosoi ingår i släktet Aechmea och familjen Bromeliaceae. Inga underarter finns listade i 
Catalogue of Life.',
            'hierarchical_section_title': 'Aechmea pimenti-velosoi',
            'page_title': 'Aechmea pimenti-velosoi'
        },
        'en': {
            'caption_title_and_reference_description': 'Aechmea pimenti-velosoi [SEP] ',
            'context_page_description': 'Aechmea pimenti-velosoi is a plant species in the genus Aechmea. It is 
endemic to the state of Santa Catarina in southern Brazil.',
            'context_section_description': 'Aechmea pimenti-velosoi is a plant species in the genus Aechmea. It is 
endemic to the state of Santa Catarina in southern Brazil.',
            'hierarchical_section_title': 'Aechmea pimenti-velosoi',
            'page_title': 'Aechmea pimenti-velosoi'
        },
        'ceb': {
            'caption_title_and_reference_description': 'Aechmea pimenti-velosoi [SEP] ',
            'context_page_description': 'Espesye sa balili ang Aechmea pimenti-velosoi. Una ning gihulagway ni 
Raulino Reitz. Ang Aechmea pimenti-velosoi sakop sa kahenera nga Aechmea sa kabanay nga Bromeliaceae. Pagka karon 
wala pay siak nga nalista ubos niini niya.',
            'context_section_description': 'Espesye sa balili ang Aechmea pimenti-velosoi. Una ning gihulagway ni 
Raulino Reitz. Ang Aechmea pimenti-velosoi sakop sa kahenera nga Aechmea sa kabanay nga Bromeliaceae. Pagka karon 
wala pay siak nga nalista ubos niini niya.',
            'hierarchical_section_title': 'Aechmea pimenti-velosoi',
            'page_title': 'Aechmea pimenti-velosoi'
        },
        'war': {
            'caption_title_and_reference_description': 'Aechmea pimenti-velosoi [SEP] ',
            'context_page_description': 'An Aechmea pimenti-velosoi in uska species han Liliopsida nga ginhulagway 
ni Raulino Reitz. An Aechmea pimenti-velosoi in nahilalakip ha genus nga Aechmea, ngan familia nga Bromeliaceae. 
Waray hini subspecies nga nakalista.',
            'context_section_description': 'An Aechmea pimenti-velosoi in uska species han Liliopsida nga 
ginhulagway ni Raulino Reitz. An Aechmea pimenti-velosoi in nahilalakip ha genus nga Aechmea, ngan familia nga 
Bromeliaceae. Waray hini subspecies nga nakalista.',
            'hierarchical_section_title': 'Aechmea pimenti-velosoi',
            'page_title': 'Aechmea pimenti-velosoi'
        },
        'az': {
            'caption_title_and_reference_description': 'Aechmea pimenti-velosoi [SEP] ',
            'context_page_description': 'Aechmea pimenti-velosoi — bromeliyakimilər fəsiləsinin exmeya cinsinə aid 
bitki növü.',
            'context_section_description': 'Aechmea pimenti-velosoi (lat. Aechmea pimenti-velosoi) —  
bromeliyakimilər fəsiləsinin exmeya cinsinə aid bitki növü.',
            'hierarchical_section_title': 'Aechmea pimenti-velosoi',
            'page_title': 'Aechmea pimenti-velosoi'
        },
        'de': {
            'caption_reference_description': 'Habitus und Blütenstand einer Aechmea purpureorosea',
            'caption_title_and_reference_description': 'Aechmea [SEP] Habitus und Blütenstand einer Aechmea 
purpureorosea',
            'context_page_description': 'Die Aechmea sind eine Pflanzengattung aus der Unterfamilie Bromelioideae 
in der Familie der Bromeliengewächse. In dieser Gattung gibt es etwa 250 bis 300 Arten. Das griechische Wort aechme
bedeutet Lanzenspitze und so findet man gelegentlich die Bezeichnung Lanze

...

{
    'wit_idx': 253147,
    'captions': {
        'en': {
            'caption_reference_description': 'The lake in front of the Monte Cerro campus',
            'caption_title_and_reference_description': 'Tamera [SEP] The lake in front of the Monte Cerro campus',
            'context_page_description': '"Tamera is a peace research village with the goal of becoming ""a 
self-sufficient, sustainable and duplicable communitarian model for nonviolent cooperation and cohabitation between
humans, animals, nature, and Creation for a future of peace for all."" It is also often called a ""healing 
biotope"". Literally translated, ""biotope"" simply means a place where life lives. In Tamera, however, ""healing 
biotope"" is also described as a ""greenhouse of trust, an acupuncture point of peace, and a self-sufficient future
community."" It is located on 335 acres in the Alentejo region of southwestern Portugal."',
            'context_section_description': 'The “SolarVillage” is a test site in Tamera creating a model village 
that will be self-sufficient in energy and food production enough for fifty people. It can be built without any 
assistance from the big energy companies. Ecology, technology, architecture, and community are combined in a 
holistic pilot project designed and monitored by physicist and inventor Jürgen Kleinwächter. With the energy 
produced by a system of lenses within a greenhouse and stored in a hot oil system, round-the-clock cooking is 
available. Power and electricity can also be produced by means of sun-powered pulses of the low-temperature 
Stirling engine. Even in a hot climate, vegetables and other useful plants flourish and grow very well under light 
filtered by these lenses.',
            'hierarchical_section_title': 'Tamera / Projects / The SolarVillage',
            'page_title': 'Tamera',
            'section_title': 'The SolarVillage'
        }
    },
    'wikipedia_caption_image': tensor([[[0.8745, 0.8745, 0.8745,  ..., 0.8510, 0.8510, 0.8510],
         [0.8745, 0.8745, 0.8745,  ..., 0.8510, 0.8510, 0.8510],
         [0.8745, 0.8745, 0.8745,  ..., 0.8510, 0.8510, 0.8510],
         ...,
         [0.4431, 0.4431, 0.4431,  ..., 0.3961, 0.3922, 0.3922],
         [0.4353, 0.4353, 0.4392,  ..., 0.3922, 0.3882, 0.3882],
         [0.4275, 0.4275, 0.4275,  ..., 0.3843, 0.3804, 0.3804]],

        [[0.9020, 0.9020, 0.9020,  ..., 0.8980, 0.8980, 0.8980],
         [0.9020, 0.9020, 0.9020,  ..., 0.8980, 0.8980, 0.8980],
         [0.9020, 0.9020, 0.9020,  ..., 0.8980, 0.8980, 0.8980],
         ...,
         [0.5216, 0.5216, 0.5216,  ..., 0.4824, 0.4784, 0.4784],
         [0.5137, 0.5137, 0.5176,  ..., 0.4784, 0.4745, 0.4745],
         [0.5059, 0.5059, 0.5059,  ..., 0.4706, 0.4667, 0.4667]],

        [[0.9412, 0.9412, 0.9412,  ..., 0.9451, 0.9451, 0.9451],
         [0.9412, 0.9412, 0.9412,  ..., 0.9451, 0.9451, 0.9451],
         [0.9412, 0.9412, 0.9412,  ..., 0.9451, 0.9451, 0.9451],
         ...,
         [0.6196, 0.6196, 0.6196,  ..., 0.5765, 0.5725, 0.5725],
         [0.6118, 0.6118, 0.6157,  ..., 0.5725, 0.5686, 0.5686],
         [0.6039, 0.6039, 0.6039,  ..., 0.5647, 0.5608, 0.5608]]]),
    'youtube_video_id': 'JzRtTDrjM0A',
    'youtube_content_video': tensor([[[[0.2353, 0.2353, 0.2369,  ..., 0.2275, 0.2260, 0.2196],
          [0.2389, 0.2389, 0.2405,  ..., 0.2310, 0.2295, 0.2232],
          [0.2431, 0.2431, 0.2448,  ..., 0.2394, 0.2371, 0.2259],
          ...,
          [0.4629, 0.5009, 0.5316,  ..., 0.7480, 0.7412, 0.7427],
          [0.4158, 0.4389, 0.5220,  ..., 0.7500, 0.7412, 0.7443],
          [0.4065, 0.4535, 0.5222,  ..., 0.7500, 0.7439, 0.7470]],

         [[0.2510, 0.2510, 0.2510,  ..., 0.2314, 0.2321, 0.2353],
          [0.2546, 0.2546, 0.2546,  ..., 0.2349, 0.2357, 0.2389],
          [0.2588, 0.2588, 0.2588,  ..., 0.2433, 0.2433, 0.2416],
          ...,
          [0.4001, 0.4222, 0.4476,  ..., 0.7212, 0.7176, 0.7192],
          [0.3537, 0.3609, 0.4390,  ..., 0.7232, 0.7176, 0.7208],
         

{
    'wit_idx': 524005,
    'captions': {
        'de': {
            'caption_reference_description': 'Fundstelle von Big Bertha, mittig im Bild',
            'context_page_description': 'Big Bertha, wissenschaftlich auch Apollo Sample 14321 genannt, ist ein auf
dem Mond gefundenes Gestein, das laut neuesten Erkenntnissen vermutlich von der Erde stammt.',
            'context_section_description': 'Am 5. Februar 1971 landete die Mannschaft von Apollo 14 im Fra Mauro, 
einem 95\xa0km großen Impaktkrater auf dem Mond. Wie Forscher zuvor schon erwartet hatten, fand man dort 
hauptsächlich Brekzie vor – also abgeschlossenes Gestein, das überwiegend aus Fragmenten anderer Gesteinsarten 
besteht. Jene Brekzien entstanden höchstwahrscheinlich durch Einschläge auf der Mondoberfläche, da die entstehende 
Hitze und Druck verschiedene Minerale zusammenpressen.',
            'hierarchical_section_title': 'Big Bertha (Mondgestein) / Geschichte',
            'page_title': 'Big Bertha (Mondgestein)',
            'section_title': 'Geschichte'
        },
        'en': {
            'caption_reference_description': 'Big Bertha on the lunar surface prior to collection. It is above the 
exact center of the photo and lies between the wheel tracks made by the  Modular Equipment Transporter (MET) or 
rickshaw-type portable workbench.',
            'context_page_description': '"Lunar Sample 14321, better known as ""Big Bertha"", is a lunar sample 
containing an embedded Earth-origin meteorite collected on the 1971 Apollo 14 mission. It was found in the Fra 
Mauro region of the Moon, and the embedded meteorite portion is the oldest known Earth rock. At 8.998 kg this 
breccia rock is the third largest Moon sample returned during the Apollo program, behind Big Muley and Great 
Scott.',
            'hierarchical_section_title': 'Big Bertha (lunar sample)',
            'page_title': 'Big Bertha (lunar sample)'
        }
    },
    'wikipedia_caption_image': tensor([[[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]],

        [[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]],

        [[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]]]),
    'youtube_video_id': 'uMg7rLP4tOY',
    'youtube_content_video': tensor([[[[0.0107, 0.0107, 0.0107,  ..., 0.0668, 0.0742, 0.0522],
          [0.0246, 0.0304, 0.0317,  ..., 0.2897, 0.2790, 0.2570],
          [0.0097, 0.0128, 0.0127,  ..., 0.3020, 0.2925, 0.2723],
          ...,
          [0.4533, 0.4533, 0.4533,  ..., 0.2891, 0.2902, 0.2917],
          [0.4492, 0.4492, 0.4492,  ..., 0.2798, 0.2798, 0.2857],
          [0.0876, 0.0876, 0.0876,  ..., 0.0541, 0.0541, 0.0573]],

         [[0.0155, 0.0155, 0.0155,  ..., 0.0261, 0.0257, 0.0191],
          [0.0903, 0.0903, 0.0918,  ..., 0.1229, 0.1169, 0.0950],
          [0.0803, 0.0803, 0.0820,  ..., 0.1245, 0.1199, 0.0997],
          ...,
          [0.3608, 0.3608, 0.3608,  ..., 0.1216, 0.1236, 0.1251],
          [0.3703, 0.3703, 0.3703,  ..., 0.1227, 0.1227,

torch.Size([160000])

{
    'wit_idx': 524005,
    'captions': {
        'de': {
            'caption_reference_description': 'Fundstelle von Big Bertha, mittig im Bild',
            'context_page_description': 'Big Bertha, wissenschaftlich auch Apollo Sample 14321 genannt, ist ein auf
dem Mond gefundenes Gestein, das laut neuesten Erkenntnissen vermutlich von der Erde stammt.',
            'context_section_description': 'Am 5. Februar 1971 landete die Mannschaft von Apollo 14 im Fra Mauro, 
einem 95\xa0km großen Impaktkrater auf dem Mond. Wie Forscher zuvor schon erwartet hatten, fand man dort 
hauptsächlich Brekzie vor – also abgeschlossenes Gestein, das überwiegend aus Fragmenten anderer Gesteinsarten 
besteht. Jene Brekzien entstanden höchstwahrscheinlich durch Einschläge auf der Mondoberfläche, da die entstehende 
Hitze und Druck verschiedene Minerale zusammenpressen.',
            'hierarchical_section_title': 'Big Bertha (Mondgestein) / Geschichte',
            'page_title': 'Big Bertha (Mondgestein)',
            'section_title': 'Geschichte'
        },
        'en': {
            'caption_reference_description': 'Big Bertha on the lunar surface prior to collection. It is above the 
exact center of the photo and lies between the wheel tracks made by the  Modular Equipment Transporter (MET) or 
rickshaw-type portable workbench.',
            'context_page_description': '"Lunar Sample 14321, better known as ""Big Bertha"", is a lunar sample 
containing an embedded Earth-origin meteorite collected on the 1971 Apollo 14 mission. It was found in the Fra 
Mauro region of the Moon, and the embedded meteorite portion is the oldest known Earth rock. At 8.998 kg this 
breccia rock is the third largest Moon sample returned during the Apollo program, behind Big Muley and Great 
Scott.',
            'hierarchical_section_title': 'Big Bertha (lunar sample)',
            'page_title': 'Big Bertha (lunar sample)'
        }
    },
    'wikipedia_caption_image': tensor([[[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]],

        [[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]],

        [[0.7412, 0.6549, 0.4784,  ..., 0.2941, 0.3490, 0.4039],
         [0.7255, 0.7412, 0.4235,  ..., 0.3765, 0.4196, 0.5216],
         [0.7176, 0.7255, 0.6902,  ..., 0.5333, 0.5490, 0.4588],
         ...,
         [0.5490, 0.7804, 0.4588,  ..., 0.1412, 0.1176, 0.2588],
         [0.5804, 0.7216, 0.5882,  ..., 0.1333, 0.1137, 0.1725],
         [0.5843, 0.7059, 0.7804,  ..., 0.1176, 0.1216, 0.1451]]]),
    'youtube_video_id': 'uMg7rLP4tOY',
    'youtube_content_video': tensor([[[[0.0107, 0.0107, 0.0107,  ..., 0.0668, 0.0742, 0.0522],
          [0.0246, 0.0304, 0.0317,  ..., 0.2897, 0.2790, 0.2570],
          [0.0097, 0.0128, 0.0127,  ..., 0.3020, 0.2925, 0.2723],
          ...,
          [0.4533, 0.4533, 0.4533,  ..., 0.2891, 0.2902, 0.2917],
          [0.4492, 0.4492, 0.4492,  ..., 0.2798, 0.2798, 0.2857],
          [0.0876, 0.0876, 0.0876,  ..., 0.0541, 0.0541, 0.0573]],

         [[0.0155, 0.0155, 0.0155,  ..., 0.0261, 0.0257, 0.0191],
          [0.0903, 0.0903, 0.0918,  ..., 0.1229, 0.1169, 0.0950],
          [0.0803, 0.0803, 0.0820,  ..., 0.1245, 0.1199, 0.0997],
          ...,
          [0.3608, 0.3608, 0.3608,  ..., 0.1216, 0.1236, 0.1251],
          [0.3703, 0.3703, 0.3703,  ..., 0.1227, 0.1227,

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://bcddf7d7fbc6cde520.gradio.live
